<a href="https://colab.research.google.com/github/Multik838/-Data-analysis-algorithms/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%20%D0%BA%20%D0%A3%D1%80%D0%BE%D0%BA%D1%83%205%20Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 34.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=63d6e454014a6d5a31bab35260b250c7d28b577c615aee0bed3968e73c8f45ef
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [3]:
from google.colab import drive
drive.mount('/content/drive')

raw_sales = spark.read.format('csv').option('header', 'True').load('/content/raw_sales.csv') 

Mounted at /content/drive


In [ ]:
#df = spark.read.csv('raw_sales.csv', inferShema=True, header=True)

In [28]:
raw_sales.show(1)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
+-------------------+--------+------+------------+--------+
only showing top 1 row



In [52]:
#Импортируем необходимые библиотеки
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import max as max_
from pyspark.sql.functions import row_number
from pyspark.sql.functions import concat

In [53]:
df = raw_sales.withColumn("id", row_number().over(Window.partitionBy().orderBy("datesold"))).alias("id")

In [54]:
df.show(2)

+-------------------+--------+------+------------+--------+---+
|           datesold|postcode| price|propertyType|bedrooms| id|
+-------------------+--------+------+------------+--------+---+
|2007-02-07 00:00:00|    2607|525000|       house|       4|  1|
|2007-02-27 00:00:00|    2906|290000|       house|       3|  2|
+-------------------+--------+------+------------+--------+---+
only showing top 2 rows



In [57]:
#Делаем конкатенационный уникальный ключ

In [69]:
df1 = df.select(F.col('datesold'),F.col('postcode'),F.col('price'),F.col('propertyType'),F.col('bedrooms'),(concat(F.col('postcode'), F.year('datesold'), F.col('bedrooms'), F.col('id')).alias('key'))).alias('key_value')

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [70]:
windSpec_10before = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 11, Window.currentRow - 1)

In [71]:
df1.withColumn('Средняя стомость 10 проданных домов до текущего в том же районе', F.avg('price').over(windSpec_10before)).show(5)

+-------------------+--------+------+------------+--------+------------+---------------------------------------------------------------+
|           datesold|postcode| price|propertyType|bedrooms|         key|Средняя стомость 10 проданных домов до текущего в том же районе|
+-------------------+--------+------+------------+--------+------------+---------------------------------------------------------------+
|2007-07-08 00:00:00|    2600|327000|       house|       1| 26002007122|                                                           null|
|2007-08-16 00:00:00|    2600|790000|       house|       4| 26002007446|                                                       327000.0|
|2007-12-05 00:00:00|    2600|825000|       house|       3|260020073129|                                                       558500.0|
|2008-01-21 00:00:00|    2600|315000|        unit|       1|260020081156|                                              647333.3333333334|
|2008-04-24 00:00:00|    2600|292500|    

In [72]:
AVGbefore = df1.withColumn('Средняя стомость 10 проданных домов до текущего в том же районе', F.avg('price').over(windSpec_10before)).alias('Before')

In [73]:
AVGbefore.show()

+-------------------+--------+-------+------------+--------+------------+---------------------------------------------------------------+
|           datesold|postcode|  price|propertyType|bedrooms|         key|Средняя стомость 10 проданных домов до текущего в том же районе|
+-------------------+--------+-------+------------+--------+------------+---------------------------------------------------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1| 26002007122|                                                           null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4| 26002007446|                                                       327000.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|260020073129|                                                       558500.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|260020081156|                                              647333.3333333334|
|2008-04-24 00:00:00|    2600| 292

In [74]:
windSpec_10after = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 10, Window.currentRow - 0)

In [76]:
df2 = AVGbefore.withColumn('Средняя стомость 10 проданных домов после текущего в том же районе', F.avg('price').over(windSpec_10after)).alias('After')

In [77]:
df2.show(10)

+-------------------+--------+-------+------------+--------+------------+---------------------------------------------------------------+------------------------------------------------------------------+
|           datesold|postcode|  price|propertyType|bedrooms|         key|Средняя стомость 10 проданных домов до текущего в том же районе|Средняя стомость 10 проданных домов после текущего в том же районе|
+-------------------+--------+-------+------------+--------+------------+---------------------------------------------------------------+------------------------------------------------------------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1| 26002007122|                                                           null|                                                          327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4| 26002007446|                                                       327000.0|                                           

In [10]:
#Ищем дату по типу собственности дом

In [78]:
filter = (df2.select('propertyType', 'datesold')\
.filter('propertyType = "house"')\
.groupBy('propertyType')
.agg(max_('datesold')))

filter.show()

+------------+-------------------+
|propertyType|      max(datesold)|
+------------+-------------------+
|       house|2019-07-27 00:00:00|
+------------+-------------------+



In [79]:
#Выводим район, цена, дата, тип собственности

In [81]:
df2.select('price', 'propertyType','datesold','postcode')\
.filter('propertyType = "house"')\
.filter('datesold = "2019-07-27 00:00:00"')\
.orderBy('postcode','price')\
.show()

+------+------------+-------------------+--------+
| price|propertyType|           datesold|postcode|
+------+------------+-------------------+--------+
|647500|       house|2019-07-27 00:00:00|    2615|
|610000|       house|2019-07-27 00:00:00|    2905|
|865000|       house|2019-07-27 00:00:00|    2905|
+------+------------+-------------------+--------+



## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [96]:
df = spark.read.csv('raw_sales.csv', inferSchema=True, header=True)

In [97]:
df = raw_sales.withColumn("id", row_number().over(Window.partitionBy().orderBy("datesold"))).alias("id")

In [98]:
df1 = df.select(F.col('datesold'),F.col('postcode'),F.col('price'),F.col('propertyType'),F.col('bedrooms'),(concat(F.col('postcode'), F.year('datesold'), F.col('bedrooms'), F.col('id')).alias('key1'))).alias('key_value')

In [130]:
windSpec_10before = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold') \
    .rowsBetween(Window.currentRow - 11, Window.currentRow - 1)

df3 = df1.withColumn('Средняя стомость 10 проданных домов до текущего в том же районе', F.avg('price').over(windSpec_10before)).groupBy(F.year('datesold').alias('mydate')).avg().alias('3')

In [131]:
df3.show()

+------+--------------------------------------------------------------------+
|mydate|avg(Средняя стомость 10 проданных домов до текущего в том же районе)|
+------+--------------------------------------------------------------------+
|  2007|                                                    528000.663522985|
|  2018|                                                   662131.1444630345|
|  2015|                                                   625357.9271082528|
|  2013|                                                   553300.4587375164|
|  2014|                                                   591913.2451338394|
|  2019|                                                   638559.3694125371|
|  2012|                                                   552709.1321352071|
|  2009|                                                    493777.933589782|
|  2016|                                                   634018.3807153263|
|  2010|                                                   55667

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [128]:
df4 = df2.select(F.col('datesold'),F.col('postcode'),F.col('price'),F.col('propertyType'),F.col('bedrooms')\
,F.col('key'),F.col('Средняя стомость 10 проданных домов до текущего в том же районе') ,F.col('Средняя стомость 10 проданных домов после текущего в том же районе') ,F.year('datesold').alias('mydata')).alias('year') 

In [129]:
df4.show(2)

+-------------------+--------+------+------------+--------+-----------+---------------------------------------------------------------+------------------------------------------------------------------+------+
|           datesold|postcode| price|propertyType|bedrooms|        key|Средняя стомость 10 проданных домов до текущего в том же районе|Средняя стомость 10 проданных домов после текущего в том же районе|mydata|
+-------------------+--------+------+------------+--------+-----------+---------------------------------------------------------------+------------------------------------------------------------------+------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|26002007122|                                                           null|                                                          327000.0|  2007|
|2007-08-16 00:00:00|    2600|790000|       house|       4|26002007446|                                                       327000.0|                         

In [116]:
df3.show(2)

+--------------+--------------------------------------------------------------------+
|year(datesold)|avg(Средняя стомость 10 проданных домов до текущего в том же районе)|
+--------------+--------------------------------------------------------------------+
|          2007|                                                    528000.663522985|
|          2018|                                                   662131.1444630345|
+--------------+--------------------------------------------------------------------+
only showing top 2 rows



In [125]:
df3.select(F.col('year(datesold)')).show(2)

+--------------+
|year(datesold)|
+--------------+
|          2007|
|          2018|
+--------------+
only showing top 2 rows



In [141]:
df7 = df4.join(df3, df3.mydate == df4.mydata, 'outer').alias('join')

In [142]:
df7.show(2)

+-------------------+--------+------+------------+--------+-----------+---------------------------------------------------------------+------------------------------------------------------------------+------+------+--------------------------------------------------------------------+
|           datesold|postcode| price|propertyType|bedrooms|        key|Средняя стомость 10 проданных домов до текущего в том же районе|Средняя стомость 10 проданных домов после текущего в том же районе|mydata|mydate|avg(Средняя стомость 10 проданных домов до текущего в том же районе)|
+-------------------+--------+------+------------+--------+-----------+---------------------------------------------------------------+------------------------------------------------------------------+------+------+--------------------------------------------------------------------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|26002007122|                                                           null|       

In [143]:
@F.udf(returnType=IntegerType())
def nunique(row):
    return len(set(row))


df7.withColumn('nunique', nunique(F.array('*'))).show()

+-------------------+--------+------+------------+--------+------------+---------------------------------------------------------------+------------------------------------------------------------------+------+------+--------------------------------------------------------------------+-------+
|           datesold|postcode| price|propertyType|bedrooms|         key|Средняя стомость 10 проданных домов до текущего в том же районе|Средняя стомость 10 проданных домов после текущего в том же районе|mydata|mydate|avg(Средняя стомость 10 проданных домов до текущего в том же районе)|nunique|
+-------------------+--------+------+------------+--------+------------+---------------------------------------------------------------+------------------------------------------------------------------+------+------+--------------------------------------------------------------------+-------+
|2007-07-08 00:00:00|    2600|327000|       house|       1| 26002007122|                                           